In [12]:
import pandas as pd
import os 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
import visdom
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
try:
    from IPython.display import display, HTML
    %load_ext autoreload
    %autoreload 2
except:
    pass

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
#!/usr/bin/env python
# coding: utf-8
import pandas as pd
import numpy as np
import os
import sys
import glob
from tqdm import tqdm
from sklearn.preprocessing import normalize

sys.path.append('./.')
sys.path.append('./..')
from pathlib import Path
import argparse
import pickle
import copy
import json
from onlineGD import onlineGD
from loss_function_grad import maxDotProd_gradient, calculate_cosineDist_gradient
from linear_model_v2 import linearClassifier_bEF

import seaborn as sns
from matplotlib import pyplot as plt
from record import record_class
import yaml
import time
from collections import OrderedDict
from common_utils import utils
from sklearn.utils import shuffle

explantions_file_path = None
embedding_data_path = None
serialID_mapping_loc = None
anomalies_pos_fpath = None
anomalies_neg_fpath = None
feedback_batch_size = None
top_K_count = None
interaction_type = 'concat'
'''
embedding_data_path  = './../../createGraph_trade/saved_model_data/{}'.format(DIR)
serialID_mapping_loc = './../../generated_data_v1/{}/idMapping.csv'.format(DIR)
anomalies_pos_fpath = './../../generated_data_v1/generated_anomalies/{}/pos_anomalies.csv'.format(DIR)
anomalies_neg_fpath = './../../generated_data_v1/generated_anomalies/{}/neg_anomalies.csv'.format(DIR)
explantions_f_path =  './../../generated_data_v1/generated_anomalies/{}/pos_anomalies_explanations.json'.format(DIR)
'''

"\nembedding_data_path  = './../../createGraph_trade/saved_model_data/{}'.format(DIR)\nserialID_mapping_loc = './../../generated_data_v1/{}/idMapping.csv'.format(DIR)\nanomalies_pos_fpath = './../../generated_data_v1/generated_anomalies/{}/pos_anomalies.csv'.format(DIR)\nanomalies_neg_fpath = './../../generated_data_v1/generated_anomalies/{}/neg_anomalies.csv'.format(DIR)\nexplantions_f_path =  './../../generated_data_v1/generated_anomalies/{}/pos_anomalies_explanations.json'.format(DIR)\n"

In [14]:
def setup_config(DIR):
    global explantions_file_path
    global embedding_data_path
    global serialID_mapping_loc
    global anomalies_pos_fpath
    global anomalies_neg_fpath
    global domain_dims
    global test_data_serialized_loc
    with open('config.yaml', 'r') as fh:
        config = yaml.safe_load(fh)

    serialID_mapping_loc = config['serialID_mapping_loc'].format(DIR)
    embedding_data_path = config['embedding_data_path'].format(DIR)
    explantions_file_path = config['explantions_file_path'].format(DIR)
    anomalies_pos_fpath = config['anomalies_pos_fpath'].format(DIR)
    anomalies_neg_fpath = config['anomalies_neg_fpath'].format(DIR)
    test_data_serialized_loc = config['test_data_serialized_loc'].format(DIR)

    with open(config['domain_dims_file_path'].format(DIR), 'rb') as fh:
        domain_dims = OrderedDict(pickle.load(fh))
    return

DIR = 'us_import4'
setup_config(DIR)

In [15]:
# ---------------------------------------------------------------------------------
def get_serialID_to_entityID():
    global serialID_mapping_loc
    idMapper_file = os.path.join(serialID_mapping_loc)
    mapping_df = pd.read_csv(idMapper_file, index_col=None)
    serialID_to_entityID = {}

    for i, row in mapping_df.iterrows():
        serialID_to_entityID[row['serial_id']] = row['entity_id']
    return serialID_to_entityID


# ---------------------------
# Get records which are deemed nominal/normal
# ---------------------------
def obtain_normal_samples():
    global test_data_serialized_loc
    normal_data = pd.read_csv(
        test_data_serialized_loc, index_col=None
    )

    _df = normal_data.sample(5000)
    obj_list = []
    for i in tqdm(range(_df.shape[0])):
        obj = record_class(_df.iloc[i].to_dict(), -1)
        obj_list.append(obj)
    data_x = []
    for _obj in obj_list:
        data_x.append(_obj.x)
    data_x = np.stack(data_x)
    return data_x


In [16]:

def get_trained_classifier(X, y, num_domains, emb_dim, num_epochs=10000):
    global domain_dims
    global interaction_type
    classifier_obj = linearClassifier_bEF(
        num_domains=num_domains,
        emb_dim=emb_dim,
        num_epochs=num_epochs,
        L2_reg_lambda=0.0025,
        force_reg=False,
        interaction_type=interaction_type
    )

    classifier_obj.setup_binaryFeatures(
        domain_dims,
        binaryF_domains=['ConsigneePanjivaID', 'ShipperPanjivaID']
    )

    # classifier_obj.fit_on_pos(X, np.ones(X.shape[0]),n_epochs=10000)
    classifier_obj.fit(X, y, log_interval=5000)
    classifier_obj.fit_on_pos(X, y, n_epochs=num_epochs // 2, log_interval=1000)
    return classifier_obj


def fetch_entityID_arr_byList(data_df, id_list):
    global domain_dims
    domain_list = list(domain_dims.keys())
    ID_COL = 'PanjivaRecordID'
    data_df = data_df.copy(deep=True)
    data_df = data_df.loc[data_df[ID_COL].isin(id_list)]
    # Order of id_list has to be preserved!!!
    X = []
    for _id in id_list:
        _tmp = data_df.loc[data_df[ID_COL] == _id][domain_list].iloc[0].values.tolist()
        X.append(_tmp)
    return np.array(X).astype(int)


In [17]:
def display_df(_df):  
    display(HTML(_df.to_html()))


In [19]:

data_source_loc = './../generated_data_v1/'
loc = os.path.join(data_source_loc, DIR)
with open(os.path.join(loc, 'domain_dims.pkl'), 'rb') as fh:
    domain_dims = OrderedDict(pickle.load(fh))

domain_list = list(domain_dims.keys())
num_domains = len(domain_dims)
domain_dims


OrderedDict([('Carrier', 604),
             ('ConsigneePanjivaID', 5979),
             ('HSCode', 125),
             ('PortOfLading', 236),
             ('PortOfUnlading', 56),
             ('ShipmentDestination', 123),
             ('ShipmentOrigin', 106),
             ('ShipperPanjivaID', 7323)])

In [33]:
interactionID2_pair = {}
k = 0 
for i in range(num_domains):
    for j in range(i+1,num_domains):
        interactionID2_pair[k] = (domain_list[i],domain_list[j])
        k+=1
            
        
def highlight_pairs(row):
    # extract pairs
    global interactionID2_pair
    exp_1 = interactionID2_pair[row['expl_1']]
    exp_2 = interactionID2_pair[row['expl_2']]
    
    color_list = []
    columns = list(OrderedDict(row.to_dict()).keys())
    valid_column2color = []
    colors = [('DarkTurquoise','DarkTurquoise'),('Coral','Coral')]
    for e, c in zip([exp_1,exp_2],colors):
        for _e,_c  in  zip(e,c):
            valid_column2color.append((_e, _c))
    _multiple_count = [ key for key,v in (Counter([ _[0] for _ in valid_column2color])).items()  if v > 1]
    _match = False
    for column in columns:
        _match = False
        for item in valid_column2color:
            if item[0] == column:
                __color_str = item[1]
                if column in _multiple_count:
                    __color_str = 'DeepPink'
                color_list.append( 'background-color: {}; font-size: 12pt'.format(__color_str ))
                _match = True
                break
        if _match == False:
            color_list.append('font-size: 11pt')
   
    return color_list

In [21]:
# tmp = all_data_df.head(5)
# exp1 = [17,10,15,12,20]
# exp2 = [17,4,5,2,10]
# tmp['expl_1'] = exp1
# tmp['expl_2'] = exp2

# tmp = tmp.style.apply(highlight_pairs, axis=1)
# tmp.hide_columns(['expl_1','expl_2'])
# # tmp.data = tmp.data[['PanjivaRecordID','Carrier','ConsigneePanjivaID','HSCode', 'PortOfUnlading','ShipmentDestination','ShipmentOrigin','ShipperPanjivaID']]
# # help(tmp)
# # tmp.table_styles
# tmp

In [22]:
# ============================================

anom_pos_df = pd.read_csv(anomalies_pos_fpath, index_col=None)
anom_neg_df = pd.read_csv(anomalies_neg_fpath, index_col=None)

# ============================================
# setup objects

serialID_to_entityID = get_serialID_to_entityID()
record_class.__setup_embedding__(embedding_data_path, serialID_to_entityID, _normalize=True)
emb_dim = record_class.embedding['HSCode'].shape[1]

# main_data_df has the records with entity ids
main_data_df = pd.concat([anom_pos_df, anom_neg_df], axis=0)
main_data_df = utils.convert_to_UnSerializedID_format(main_data_df, DIR)
# -------------------------------------------
obj_list = []
for i in tqdm(range(anom_neg_df.shape[0])):
    obj = record_class(anom_neg_df.iloc[i].to_dict(), -1)
    obj_list.append(obj)

for i in tqdm(range(anom_pos_df.shape[0])):
    obj = record_class(anom_pos_df.iloc[i].to_dict(), 1)
    obj_list.append(obj)

# Read in the explantions
with open(explantions_file_path, 'rb') as fh:
    explanations = json.load(fh)

explanations = {int(k): [sorted(_) for _ in v] for k, v in explanations.items()}
num_domains = len(domain_dims)
domain_idx = {e[0]: e[1] for e in enumerate(domain_dims.keys())}

domainInteraction_index = {}
k = 0
for i in range(num_domains):
    for j in range(i + 1, num_domains):
        domainInteraction_index['_'.join((domain_idx[i], domain_idx[j]))] = k
        k += 1

data_x = []
data_id = []
data_label = []
data_ID_to_matrix = {}

for _obj in obj_list:
    data_x.append(_obj.x)
    data_id.append(_obj.id)
    data_label.append(_obj.label)
    data_ID_to_matrix[_obj.id] = _obj.x

data_x = np.stack(data_x)
data_label = np.array(data_label)
data_id = np.array(data_id)

idx = np.arange(len(data_id), dtype=int)
np.random.shuffle(idx)

data_x = data_x[idx]
data_label = data_label[idx]
data_id = data_id[idx]

X_0 = data_x  # Relevant anomalies
X_1 = obtain_normal_samples()  # Nominal
y_0 = np.ones(X_0.shape[0])
y_1 = -1 * np.ones(X_1.shape[0])
y = np.hstack([y_0, y_1])
X = np.vstack([X_0, X_1])
num_coeff = len(domainInteraction_index)
classifier_obj = get_trained_classifier(X, y, num_domains, emb_dim)
W = classifier_obj.W.cpu().data.numpy()
emb_dim = W.shape[-1]

# classifier_obj.predict_score_op(X_0)
# Create a reference dataframe  :: data_reference_df
data_reference_df = pd.DataFrame(
    data=np.vstack([data_id, data_label]).transpose(),
    columns=['PanjivaRecordID', 'label']
)

data_reference_df['baseID'] = data_reference_df['PanjivaRecordID'].apply(lambda x: str(x)[:-3])
data_reference_df['expl_1'] = -1
data_reference_df['expl_2'] = -1
data_reference_df['original_score'] = 1

for i, row in data_reference_df.iterrows():
    _id = int(row['PanjivaRecordID'])
    if _id in explanations.keys():
        entry = explanations[_id]
        domain_1 = entry[0][0]
        domain_2 = entry[0][1]
        data_reference_df.loc[i, 'expl_1'] = domainInteraction_index['_'.join(sorted([domain_1, domain_2]))]
        domain_1 = entry[1][0]
        domain_2 = entry[1][1]
        data_reference_df.loc[i, 'expl_2'] = domainInteraction_index['_'.join(sorted([domain_1, domain_2]))]
    _x = data_ID_to_matrix[_id]
    data_reference_df.loc[i, 'original_score'] = classifier_obj.predict_score_op(np.array([_x]))[0]

data_reference_df['cur_score'] = data_reference_df['original_score'].values

  0%|          | 49/10000 [00:00<00:39, 255.02it/s]

Step 1 Loss 0.6215


 50%|█████     | 5040/10000 [00:16<00:16, 294.40it/s]

Step 5001 Loss 0.0948


  1%|          | 33/5000 [00:00<00:15, 323.32it/s]

...

 61%|██████    | 3046/5000 [00:09<00:05, 348.22it/s]

Step 3001 Loss 0.0001


 81%|████████  | 4040/5000 [00:11<00:02, 343.40it/s]

Step 4001 Loss 0.0003


100%|██████████| 5000/5000 [00:14<00:00, 336.70it/s]


In [23]:
import os
import pathlib
import numpy as np
import random
fakeName_dict = {}

def create_fakename_mapping(attribute, count):
    _attr = ''.join([char for char in attribute if char.isupper()] )
    cur_path  = './tmp'

    with open(os.path.join(cur_path, 'colors.txt'), 'r') as f:
        _colors_ = [str(_).strip() for _ in f.readlines()]

    with open(os.path.join(cur_path, 'usernames.txt'), 'r', encoding="latin-1") as f:
        _words_ =  [ str(_).strip() for _ in f.readlines() if len(_)>4  and len(_) < 12]
    np.random.shuffle(_colors_)
    np.random.shuffle(_words_)
    _u = np.random.choice(_colors_, size = count, replace=True )
    _v = np.random.choice(_words_, size = count, replace=False )
    map_list = {}
    i = 0
    for a,b in zip(_u,_v):
        map_list[i] = str(_attr) + '-' + a.title() + b.title()
        i += 1
    return map_list


def setup_FakeNameSetup(domain_dims):
    global fakeName_dict 
    fakeName_dict = {}
    for dom,dim in domain_dims.items():
        fakeName_dict[dom] = create_fakename_mapping(dom,dim)
    return 

setup_FakeNameSetup(domain_dims)

In [24]:
def execute_with_input(
        clf_obj,
        working_df,
        ref_data_df,
        domainInteraction_index,
        num_coeff,
        emb_dim,
        data_ID_to_matrix,
        check_next_value=20,
        batch_size=10, 
        max_iter = None
):
    global domain_dims
    global interaction_type
    ID_COL = 'PanjivaRecordID'
    BATCH_SIZE = batch_size
    working_df['delta'] = 0
    OGD_obj = onlineGD(num_coeff, emb_dim, calculate_cosineDist_gradient, interaction_type = interaction_type)
    W = clf_obj.W.cpu().data.numpy()
    OGD_obj.set_original_W(W)

    max_num_batches = len(working_df) // BATCH_SIZE + 1
    if max_iter is not None:
        max_num_batches = max_iter
    acc = []
    recall = []
    domain_list = list(domain_dims.keys())
    discovered_count = [0.0]
    total_relCount = len(working_df.loc[working_df['label'] == 1])

    # -------------------------------------------------
    #  Main loop
    # -------------------------------------------------
    next_K_precision = []
    W_list = []
    
    for batch_idx in tqdm(range(max_num_batches)):
         
        cur = working_df.head(BATCH_SIZE)
#         print(len(cur))
        display_details(cur.loc[cur['label']==1])
        flags = []  # Whether a pos anomaly or not
        terms = []  # Explanation terms

        # Count( of discovered in the current batch ( at the top; defined by batch size )
        cum_cur_discovered = discovered_count[-1] + len(cur.loc[cur['label'] == 1])
        _recall = float(cum_cur_discovered) / total_relCount
        discovered_count.append(
            cum_cur_discovered
        )
        recall.append(_recall)
        
        
        x_ij = []
        x_entityIds = []
        W_list.append(clf_obj.W.cpu().data.numpy())
        for i, row in cur.iterrows():
            _mask = np.zeros(len(domainInteraction_index))
            if row['label'] == 1:
                _mask[row['expl_1']] = 1
                _mask[row['expl_2']] = 1
                flags.append(1)
                terms.append((row['expl_1'], row['expl_2'],))
            else:
                flags.append(0)
                terms.append(())
            id_value = row['PanjivaRecordID']
            x_ij.append(data_ID_to_matrix[id_value])

            row_dict = ref_data_df.loc[(ref_data_df[ID_COL] == id_value)].iloc[0].to_dict()
            x_entityIds.append([row_dict[d] for d in domain_list])

        x_entityIds = np.array(x_entityIds)
        x_ij = np.array(x_ij)
        
        
        final_gradient, _W = OGD_obj.update_weight(
            flags,
            terms,
            x_ij
        )
        # ----------------------------------------------------
        # Update Model
        # ----------------------------------------------------
        clf_obj.update_W(_W)
        clf_obj.update_binary_VarW(x_entityIds, flags)

        _tail_count = len(working_df) - BATCH_SIZE
        working_df = working_df.tail(_tail_count).reset_index(drop=True)

        if len(working_df) == 0:
            break

        # Obtain scores
        x_ij_test = []
        x_entityIds = fetch_entityID_arr_byList(
            ref_data_df,
            working_df['PanjivaRecordID'].values.tolist()
        )
        for _id in working_df['PanjivaRecordID'].values:
            x_ij_test.append(data_ID_to_matrix[_id])

        x_ij_test = np.array(x_ij_test)
        new_scores = clf_obj.predict_bEF(x_entityIds, x_ij_test)
        old_scores = working_df['cur_score'].values
        _delta = new_scores - old_scores
        working_df['delta'] = _delta
        working_df = working_df.sort_values(by='delta', ascending=False)
        working_df = working_df.reset_index(drop=True)
        
        check_next = check_next_value 
        tmp = working_df.head(check_next)
        minMax_scaler_obj = MinMaxScaler()
        tmp.loc[:,'delta'] = minMax_scaler_obj.fit_transform(tmp['delta'].values.reshape([-1,1]))
        
          
        _labels = tmp['label'].values
        res = len(np.where(_labels == 1)[0])
        _precision = res / check_next
        print(_precision)
        
        
    return W_list

In [25]:
all_data_df = pd.DataFrame(anom_pos_df).copy().append(anom_neg_df,ignore_index=True)

In [31]:
def display_details(_df):
    global all_data_df
    global domain_dims
    global fakeName_dict
    global serialID_to_entityID
    viz_cols = ['PanjivaRecordID'] + list(domain_dims.keys()) + ['delta']
    _hide_cols = [ _ for _ in list(_df.columns) if _ not in viz_cols ]
    _df = _df.merge( all_data_df, on='PanjivaRecordID', how='left')
    
    
    # Replace IDs with fake names
    def _replace_wfakeName(row):
       
        for key in fakeName_dict.keys():
         
            row[key] = fakeName_dict[key][serialID_to_entityID[row[key]]]
        return row
    
    _df = _df.apply(_replace_wfakeName, axis=1) 
        
    _df = _df.style.apply(highlight_pairs,axis=1)
    _df = _df.hide_columns(_hide_cols)
    display(_df)
    return

In [39]:
# To get random results
# Randomization
cur_df = data_reference_df.copy()
cur_df = cur_df.sample(frac=1).reset_index(drop=True)
cur_df = shuffle(cur_df).reset_index(drop=True)

r = execute_with_input(
    clf_obj=copy.deepcopy(classifier_obj),
    working_df=cur_df,
    ref_data_df=main_data_df,
    domainInteraction_index=domainInteraction_index,
    num_coeff=num_coeff,
    emb_dim=emb_dim,
    data_ID_to_matrix=data_ID_to_matrix,
    batch_size=10,
    check_next_value = 10,
    max_iter = 3
)

  0%|          | 0/3 [00:00<?, ?it/s]

,PanjivaRecordID,delta,Carrier,ConsigneePanjivaID,HSCode,PortOfLading,PortOfUnlading,ShipmentDestination,ShipmentOrigin,ShipperPanjivaID
0,1289633630035001,0,C-EmeraldAdalia,CPID-VioletRoaring,HSC-CelesteHollie,POL-EmeraldChadd,POU-WhiteSomehow,SD-BoneSifuentes,SO-CitronFranglais,SPID-GoldHousewife
1,1248123090039001,0,C-CitronSaenz,CPID-CoffeeCitarella,HSC-CitronElfreda,POL-EmeraldChadd,POU-BordeauxWhicker,SD-BoneSifuentes,SO-HeatherElston,SPID-IvorySundried
2,1268800880012002,0,C-RustMcgrody,CPID-LimeGuimond,HSC-CelesteDatha,POL-ChocolateInsidious,POU-MagentaAfton,SD-GrayAnson,SO-LimeCrispi,SPID-ChocolateChristie
3,1245719520020004,0,C-DenimAeromancy,CPID-MintApogee,HSC-CelesteHollie,POL-BlueLazarus,POU-JetTricotine,SD-EbonySaros,SO-CoalObsecrate,SPID-ChocolateChristie
4,1276880660032001,0,C-CoalJaynajayne,CPID-AsphaltMolar,HSC-CoralAnstus,POL-ChocolateRico,POU-ChampagneMantling,SD-BoneSifuentes,SO-CoalObsecrate,SPID-MaroonLight


 33%|███▎      | 1/3 [00:00<00:01,  1.12it/s]

1.0


,PanjivaRecordID,delta,Carrier,ConsigneePanjivaID,HSCode,PortOfLading,PortOfUnlading,ShipmentDestination,ShipmentOrigin,ShipperPanjivaID
0,1268800880012003,1.000000,C-RustMcgrody,CPID-LimeGuimond,HSC-BeigeTreat,POL-ChocolateInsidious,POU-PurpleStubbs,SD-GrayAnson,SO-LimeCrispi,SPID-ChocolateChristie
1,124571952005004,0.627924,C-DenimAeromancy,CPID-MintApogee,HSC-CelesteHollie,POL-PurpleGayelord,POU-MagentaAfton,SD-OrangeAuthority,SO-AquamarineHubble,SPID-ChocolateChristie
2,1268800880012001,0.575710,C-RustMcgrody,CPID-LimeGuimond,HSC-CelesteDatha,POL-ChocolateInsidious,POU-MaroonHauler,SD-CreamReagan,SO-PurpleOlvera,SPID-ChocolateChristie
3,1268800880012004,0.572218,C-RustMcgrody,CPID-LimeGuimond,HSC-ChestnutArdenia,POL-ChocolateInsidious,POU-MagentaAfton,SD-GrayAnson,SO-CeruleanArlie,SPID-ChocolateChristie
4,1245719520020002,0.441610,C-DenimAeromancy,CPID-MintApogee,HSC-AshRhinestone,POL-BlueLazarus,POU-MagentaAfton,SD-CitronTruong,SO-SilverAirlie,SPID-ChocolateChristie
5,1245719520020001,0.347136,C-DenimAeromancy,CPID-MintApogee,HSC-AshRhinestone,POL-BlueLazarus,POU-MaroonHauler,SD-NavyPolemist,SO-CoalObsecrate,SPID-ChocolateChristie
6,1276880660032002,0.317706,C-CoalJaynajayne,CPID-AsphaltMolar,HSC-CoralAnstus,POL-SteelUnearth,POU-BordeauxWhicker,SD-BoneSifuentes,SO-RustShoot,SPID-MaroonLight
7,124571952005002,0.279284,C-DenimAeromancy,CPID-MintApogee,HSC-AzureBirecree,POL-EbonyOfficious,POU-MagentaAfton,SD-OrangeAuthority,SO-KhakiSparrow,SPID-ChocolateChristie
8,1245719520020003,0.252887,C-DenimAeromancy,CPID-MintApogee,HSC-JetSelina,POL-BlueLazarus,POU-MagentaAfton,SD-CitronTruong,SO-EmeraldLazar,SPID-ChocolateChristie
9,1248237860034003,0.000000,C-RustMcgrody,CPID-EbonyCorene,HSC-CelesteHollie,POL-ChocolateInsidious,POU-AmberCurtsy,SD-CaramelRoughdry,SO-CoalObsecrate,SPID-CamelFourinhand


 67%|██████▋   | 2/3 [00:01<00:00,  1.06it/s]

1.0


,PanjivaRecordID,delta,Carrier,ConsigneePanjivaID,HSCode,PortOfLading,PortOfUnlading,ShipmentDestination,ShipmentOrigin,ShipperPanjivaID
0,124571952005001,1.000000,C-LimeKarr,CPID-MintApogee,HSC-AzureBirecree,POL-BlueLazarus,POU-MagentaAfton,SD-OrangeAuthority,SO-BlueBarton,SPID-ChocolateChristie
1,1276880660028003,0.797469,C-PurpleSweater,CPID-AsphaltMolar,HSC-AshRhinestone,POL-EmeraldChadd,POU-BordeauxWhicker,SD-BoneSifuentes,SO-WhiteAbsinthism,SPID-MaroonLight
2,1268779670022004,0.793346,C-RustMcgrody,CPID-LimeGuimond,HSC-MaroonDori,POL-ChocolateInsidious,POU-CamelCanakin,SD-GrayAnson,SO-LimeCrispi,SPID-CamelFourinhand
3,1250458490081002,0.674778,C-BisqueAlfalfa,CPID-AsphaltMolar,HSC-AshRhinestone,POL-EmeraldChadd,POU-BordeauxWhicker,SD-CelesteIndre,SO-AsphaltBaguette,SPID-TomatoRiedel
4,1248237860034004,0.643375,C-RustMcgrody,CPID-EbonyCorene,HSC-AzureBirecree,POL-ChocolateInsidious,POU-JetRibbon,SD-IndigoHiga,SO-CoalObsecrate,SPID-CamelFourinhand
5,1248237860034002,0.415387,C-RustMcgrody,CPID-EbonyCorene,HSC-AshRhinestone,POL-ChocolateInsidious,POU-MaroonHauler,SD-CaramelRoughdry,SO-TomatoPlutonic,SPID-CamelFourinhand
6,1250458490081004,0.346608,C-BisqueAlfalfa,CPID-AsphaltMolar,HSC-JetSelina,POL-VioletBackfill,POU-BordeauxWhicker,SD-BoneSifuentes,SO-BlueBarton,SPID-TomatoRiedel
7,1250458490081001,0.304755,C-OrangeFitting,CPID-AsphaltMolar,HSC-AshRhinestone,POL-EmeraldChadd,POU-BordeauxWhicker,SD-BoneSifuentes,SO-BlueAbisia,SPID-TomatoRiedel
8,1276880660028001,0.228117,C-SnowOlibanum,CPID-AsphaltMolar,HSC-CeruleanArrearage,POL-EmeraldChadd,POU-BlackBeeler,SD-BoneSifuentes,SO-CoalObsecrate,SPID-MaroonLight
9,1276880660028002,0.000000,C-PurpleSweater,CPID-AsphaltMolar,HSC-CeruleanArrearage,POL-EmeraldChadd,POU-BordeauxWhicker,SD-BoneSifuentes,SO-PinkRegimen,SPID-MaroonLight


100%|██████████| 3/3 [00:02<00:00,  1.07it/s]

1.0


In [81]:
for _ in r :
    print( np.linalg.norm(_,axis=1))

[0.07550237 0.0716181  0.07402188 0.10690208 0.07411554 0.08524173
 0.07852211 0.08150564 0.08366144 0.11341554 0.08414254 0.09280898
 0.08762005 0.08016008 0.11075076 0.08007225 0.09162906 0.08425989
 0.11225653 0.08265345 0.09182195 0.0863458  0.11330314 0.1201796
 0.11532192 0.09453653 0.08628307 0.09631667]
[0.14268766 0.0716181  0.07402188 0.10690208 0.07411554 0.08524173
 0.16469519 0.13325967 0.15689152 0.11341554 0.08414254 0.14202096
 0.08762005 0.08016008 0.11075076 0.08007225 0.09162906 0.08425989
 0.11225653 0.08265345 0.09182195 0.0863458  0.11330314 0.1201796
 0.16459468 0.09453653 0.08628307 0.09631667]
[0.18220706 0.0716181  0.07402188 0.10690208 0.07411554 0.08524173
 0.16469519 0.16706431 0.20565118 0.11341554 0.08414254 0.1807607
 0.08762005 0.08016008 0.11075076 0.08007225 0.09162906 0.08425989
 0.11225653 0.08265345 0.09182195 0.0863458  0.11330314 0.1201796
 0.16459468 0.09453653 0.08628307 0.09631667]


In [34]:
data_ID_to_matrix[125135714001].shape

(8, 128)